# Web Scraping

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

list1=[]
# Path ke Chrome Driver

def scraping_yt(i):
    driver_path = '/path/to/chromedriver'

    # URL video YouTube
    video_url = f'https://www.youtube.com/watch?v={i}'

    # Buat objek WebDriver
    driver = webdriver.Chrome(driver_path)

    # Buka halaman video
    driver.get(video_url)

    # Scroll halaman untuk memuat lebih banyak komentar
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Ambil HTML halaman web saat ini
    page_html = driver.page_source

    # Buat objek BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')

    # Cari elemen yang berisi komentar
    comment_elements = soup.find_all('yt-formatted-string', {'class': 'style-scope ytd-comment-renderer'})

    # Loop melalui elemen komentar dan cetak isi komentar
    for comment in comment_elements:
        list1.append(comment.text)

    driver.quit()

    return list1

video_id = ['4sMQiuKKU4Y','fj6Lxd2XMkw','Pp_35yWH8Bk','TZnM7SZyU18','dxSiy1vKZCU']
for i in video_id:
    scraping_yt(i)

# Masukkan Data dalam csv

In [ ]:
import pandas as pd

def list_to_csv(list_data, filename):
    df = pd.DataFrame(list_data, columns=['Kalimat'])
    df.to_csv(filename, index=False)

# Contoh penggunaan
list_to_csv(list1, 'comment.csv')


# Membuat Session

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=9dafc17e1cdecf13f82bf37174c97ce251059b978655dd1771131d8546095c62
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
#mengimport modul yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

#membuat session
appName = "Sentiment Analysis di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Import Data

In [ ]:
from google.colab import files

# Mengunggah file dari direktori lokal
uploaded = files.upload()


Saving comment.csv to comment.csv


In [ ]:
# Baca file CSV ke DataFrame
df = spark.read.csv("comment.csv", header=True, inferSchema=True)


In [ ]:
df.show(truncate=False,n=3)

+--------------------------------------------------------------------------------------------+
|Kalimat                                                                                     |
+--------------------------------------------------------------------------------------------+
|Akhirnya sudah kebeli tiket ultimate coldplay. Can't wait in November 2023!!                |
|Aku semua genre suka,,dr band Coldplay cuma lagu paradise, yellow aja yg lumayan aku suka,,,|
|null                                                                                        |
+--------------------------------------------------------------------------------------------+
only showing top 3 rows



In [ ]:
#menghitung baris dari DataFrame kita, dan menge-print untuk menampilkan
jumlahBaris = df.count()
print("jumlah baris: ", jumlahBaris)
#melihat skema dari dataframe kita
df.printSchema()


jumlah baris:  236
root
 |-- Kalimat: string (nullable = true)



In [ ]:
# Menghapus baris yang mengandung nilai null
df_no_null = df.dropna()

# Menampilkan DataFrame hasil
jumlahBaris = df_no_null.count()
print("jumlah baris: ", jumlahBaris)
df.show(truncate=False,n=3)

jumlah baris:  232
+--------------------------------------------------------------------------------------------+
|Kalimat                                                                                     |
+--------------------------------------------------------------------------------------------+
|Akhirnya sudah kebeli tiket ultimate coldplay. Can't wait in November 2023!!                |
|Aku semua genre suka,,dr band Coldplay cuma lagu paradise, yellow aja yg lumayan aku suka,,,|
|null                                                                                        |
+--------------------------------------------------------------------------------------------+
only showing top 3 rows



# Pre-processing

## Case Folding 

In [ ]:
# Melakukan case folding pada kolom "Komentar"
df_lower = df_no_null.withColumn("Kalimat", lower(col("Kalimat")))

# Menampilkan DataFrame hasil case folding
df_lower.show(truncate=False, n=232)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Kalimat                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
+-----------------------------------

## Remove Punctuation 

In [ ]:
# Melakukan penghapusan tanda baca pada kolom "Komentar"
df_no_punct = df_lower.withColumn("Kalimat", regexp_replace(col("Kalimat"), "[^\w\s]", ""))

# Menampilkan DataFrame hasil penghapusan tanda baca
df_no_punct.show(truncate=False, n=232)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Kalimat                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+-------------------------------------------------------------------------------------

## Repeated Character

In [ ]:
# Melakukan penghapusan karakter berulang pada kolom "Text"
df_no_repeated = df_no_punct.withColumn("Kalimat", regexp_replace(col("Kalimat"), r"(.)\1+", r"\1"))

# Menampilkan DataFrame hasil penghapusan karakter berulang
df_no_repeated.show(truncate=False, n=232)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Kalimat                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
+---------------------------------------------------------------------------------------

In [ ]:
df_no_numerik = df_no_repeated.withColumn("Kalimat", regexp_replace(df_no_repeated["Kalimat"], "[^a-zA-Z0-9\\s]", ""))  # Hapus karakter non-alfanumerik
df_no_numerik.show(truncate=False, n=232)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Kalimat                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
+---------------------------------------------------------------------------------------

## Tokenization 

In [ ]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Kalimat", outputCol="kal_token")
tokenizedTrain = tokenizer.transform(df_no_numerik)
tokenizedTrain.show(truncate=False, n=232)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Kalimat       

## StopWords

In [ ]:
# Mengunggah file dari direktori lokal
upload = files.upload()

Saving stopwords-id.txt to stopwords-id.txt


In [ ]:
stopwords_df = spark.read.text("stopwords-id.txt")
stopwords_list = [row['value'] for row in stopwords_df.collect()]

In [ ]:
remover = StopWordsRemover(inputCol='kal_token', outputCol='filtered_text', stopWords=stopwords_list)

# Transformasikan DataFrame dengan menghapus stopwords
filtered_df = remover.transform(tokenizedTrain)

# Tampilkan hasil
filtered_df.show(truncate=False,n=232)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------

## Lemmatization

In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from nltk.stem import WordNetLemmatizer

# Define a user-defined function (UDF) for lemmatization
def lemmatize_text(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

# Register the UDF
lemmatize_udf = udf(lemmatize_text, ArrayType(StringType()))

# Apply lemmatization using the UDF
lemmatized_data = filtered_df.withColumn("lemmas", lemmatize_udf("filtered_text"))


# Show the lemmatized data
lemmatized_data.show(truncate=False, n=232)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------

In [ ]:
# Menampilkan kolom "age"
lemmatized_data.select("lemmas").show(truncate=False, n=232)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|lemmas                                                                                                                                                                                                                                                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Lexicon Based

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from nltk.sentiment import SentimentIntensityAnalyzer


# Membuat objek SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Fungsi untuk menentukan sentimen berdasarkan compound_score
def determine_sentiment(lemmas):
    if len(lemmas) == 0:
        return 'netral'
    
    sentiment_score = 0
    for lemma in lemmas:
        sentiment_score += sia.polarity_scores(lemma)['compound']
    sentiment_score /= len(lemmas)
    
    if sentiment_score > 0.00:
        return 'positif'
    elif sentiment_score < 0.00:
        return 'negatif'
    else:
        return 'netral'

# Mendefinisikan UDF (User-Defined Function)
determine_sentiment_udf = udf(determine_sentiment, StringType())

# Menambahkan kolom 'sentimen' berdasarkan compound_score
df1 = lemmatized_data.withColumn('sentimen', determine_sentiment_udf(lemmatized_data['lemmas']))

# Menampilkan DataFrame dengan kolom 'sentimen'
df1.select("lemmas", 'sentimen').show(truncate=False, n=232)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|lemmas                                                                                                                                                                                                                                                                                                                                                  |sentimen|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
kamus_sentimen = {
    1: ["bagus", "baik", "keren", "jenis", "asik", "sukses", "favorit", "cocok", "populer",
                  "terkenal", "cinta", "syukur", "positif", "senang", "bahagia", "karya", "lagu", "lancar",
                  "seru", "musisi", "antusias", "cinta", "harapan", 'fan','penggemar','fans','ikut','nonton'
                  'datang','','ramah','asyik','ingin','kepengen','sayang','great','thank','love','seru','kagum'],
    2: ["Pelangi", "pelangi", "jelek", "lgbt", "LGBT", "batal", "fomo", "gay", "gagal",
                  "mampus", "hangus", "rusak", "aktivis", "kotor", "mahal", "kecewa", "munafik",
                  "hanguskan", "tolak", "batalkan", "isu", "kontra", "demo", "hancur", "sia", "tua",
                  "barat", "halangi", "ribut",'jgn','mahal','standar','pusing','repot','hoax','buang',
                'tolak','sok','jijik','sampah','gagal','mati','batalkan','menolak','menghalangi','mabuk',
                'kapir','kaum','ga','da1al','matrek','ketebak'],
}

# Fungsi untuk menentukan sentimen berdasarkan lemmas
def determine_sentiment(lemmas):
    for sentimen, kata_kunci in kamus_sentimen.items():
        for kata in kata_kunci:
            if kata in lemmas:
                return sentimen
    return 0

# Mendefinisikan UDF (User-Defined Function)
determine_sentiment_udf = udf(determine_sentiment, StringType())

# Menambahkan kolom 'sentimen' berdasarkan lemmas
df1 = lemmatized_data.withColumn('sentimen', determine_sentiment_udf(lemmatized_data['lemmas']))

# Menampilkan DataFrame dengan kolom 'sentimen'
df1.select("lemmas",'sentimen').show(truncate=False, n=232)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|lemmas                                                                                                                                                                                                                                                                                                                                                  |sentimen|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_B= df1.select('sentimen')


In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

df_A = df.withColumn("index", monotonically_increasing_id())
df_B = df_B.withColumn("index", monotonically_increasing_id())

# Menggabungkan DataFrame A dan B berdasarkan kolom indeks
df_joined = df_A.join(df_B, "index", "outer").drop("index")
df_joined.show(truncate=False, n=232)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Kalimat                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |sentimen|
+-----------------

In [ ]:
data_final = df1.select("Kalimat",'sentimen')
data_final.show(truncate=False, n=3)

+-----------------------------------------------------------------------------------------------------+--------+
|Kalimat                                                                                              |sentimen|
+-----------------------------------------------------------------------------------------------------+--------+
|akhirnya sudah kebeli tiket ultimate coldplay cant wait in november 2023                             |0       |
|aku semua genre sukadr band coldplay cuma lagu paradise ye1ow aja yg lumayan aku suka                |1       |
|jgn ada kerusuhan pokoknya gw nonton coldplay1band era 21 an yg paling oke lirik lagu gw suka banget |1       |
+-----------------------------------------------------------------------------------------------------+--------+
only showing top 3 rows



In [ ]:
data_final.printSchema()


root
 |-- Kalimat: string (nullable = true)
 |-- sentimen: string (nullable = true)



In [ ]:
# Konversi tipe data kolom "age" dari string ke integer
data_final = data_final.withColumn("sentimen", col("sentimen").cast("integer"))
data_final.printSchema()


root
 |-- Kalimat: string (nullable = true)
 |-- sentimen: integer (nullable = true)



# Sentiment Analysis

In [ ]:
import pyspark.sql.functions as F
import plotly.graph_objects as go

# Menghitung jumlah masing-masing sentimen
sentimen_counts = df1.groupBy('sentimen').count().collect()

# Membuat list untuk menyimpan jumlah sentimen dan label
labels = [row['sentimen'] for row in sentimen_counts]
values = [row['count'] for row in sentimen_counts]

# Membuat Pie Chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

# Menampilkan Pie Chart
fig.show()


## Split Data

In [ ]:
dataTerpisah = data_final.randomSplit([0.7, 0.3])
train = dataTerpisah[0]
#pada data testing, kita rename labelnya dari "label" ke "trueLabel"
test = dataTerpisah[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Jumlah baris data training:", train_rows, ", jumlah baris data testing:", test_rows)

Jumlah baris data training: 162 , jumlah baris data testing: 70


## Menyiapkan Data Testing

In [ ]:
tokenizer = Tokenizer(inputCol="Kalimat", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(train)
tokenizedTrain.show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------+--------+-----------------------------------------------------------------------------------------------------------------+
|Kalimat                                                                                         |sentimen|SentimentWords                                                                                                   |
+------------------------------------------------------------------------------------------------+--------+-----------------------------------------------------------------------------------------------------------------+
|                                                                                                |0       |[]                                                                                                               |
|                                                                                                |0       |[]   

In [ ]:
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords", locale="en")

SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------+--------+-----------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|Kalimat                                                                                         |sentimen|SentimentWords                                                                                                   |MeaningfulWords                                                                                                  |
+------------------------------------------------------------------------------------------------+--------+-----------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------

In [ ]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrain = hashTF.transform(SwRemovedTrain).select(
    'Kalimat','sentimen', 'MeaningfulWords', 'features')
numericTrain.show(truncate=False, n=3)

+-------------------------------------------------+--------+----------------------------------------------------------+------------------------------------------------------------------------------------------------+
|Kalimat                                          |sentimen|MeaningfulWords                                           |features                                                                                        |
+-------------------------------------------------+--------+----------------------------------------------------------+------------------------------------------------------------------------------------------------+
|                                                 |0       |[]                                                        |(262144,[249180],[1.0])                                                                         |
|                                                 |0       |[]                                                        |(262144,[2491

## Training Model

In [ ]:
lr = LogisticRegression(labelCol="sentimen", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrain)
print ("Training selesai!")

Training selesai!


## Persiapan data testing

In [ ]:
tokenizedTest = tokenizer.transform(test)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Kalimat','sentimen', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)

+--------------------------------------------------------------+--------+-----------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+
|Kalimat                                                       |sentimen|MeaningfulWords                                                        |features                                                                                         |
+--------------------------------------------------------------+--------+-----------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+
| kalau memakai teknologi internet buatan kapir                |2       |[, kalau, memakai, teknologi, internet, buatan, kapir]                 |(262144,[67569,105296,153250,168380,249180,253695,255567],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])         |
|alasa1ya promotor bingu

## Prediksi

In [ ]:
prediksiMentah = model.transform(numericTest)
prediksiFinal = prediksiMentah.select(
    'Kalimat', "prediction", "sentimen")
prediksiFinal.show(n=40, truncate = False)
prediksiBenar = prediksiFinal.filter(
    prediksiFinal['prediction'] == prediksiFinal['sentimen']).count()
totalData = prediksiFinal.count()
print("prediksi benar: ", prediksiBenar, ", total data: ", 
      totalData, ", akurasi: ", prediksiBenar/totalData)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+
|Kalimat                                                                                                                                                                                                                   |prediction|sentimen|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+
| kalau memakai teknologi internet buatan kapir                                                                                                                                                                            |0.0       |2       |
|alasa1ya promotor bingung nampung j